<a href="https://colab.research.google.com/github/davidmonterocrespo24/test_ai/blob/main/example_colab_google.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para entrenar un modelo de lenguaje GPT-2 en Python usando el código de una carpeta, necesitarás seguir los siguientes pasos:

Instalar las dependencias necesarias:

In [ ]:
!pip install tensorflow
!pip install tensorflow_datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:

import tensorflow as tf
import tensorflow_datasets as tfds
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
import os


path = 'sample_data'
text = ''
for filename in os.listdir(path):
    if not os.path.isdir(filename):
        print("##########"+str(filename))
        try:
          with open(os.path.join(path, filename), 'rb') as f:
                  text += f.read().decode()
        except Exception as e:
          print("##########"+str(e))
print(len(text))


##########archivo_modelo_odoo.txt
1065354


In [52]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
#model.config.max_position_embeddings = 997817 
tokenizer.model_max_length = 997817 
tokenized_text = tokenizer.encode(text)
examples = []
block_size = 100
BATCH_SIZE = 12
BUFFER_SIZE = 1000
for i in range(0, len(tokenized_text)-block_size+1, block_size):
    examples.append(tokenized_text[i:i+block_size])
inputs, labels = [], []
for ex in examples:
    inputs.append(ex[:-1])
    labels.append(ex[1:])
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
model = TFGPT2LMHeadModel.from_pretrained('gpt2', pad_token_id=tokenizer.eos_token_id)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), loss=[loss_fn, *[None] * model.config.n_layer])



All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [53]:
EPOCHS = 1
model.fit(dataset, epochs=EPOCHS)
model.save_pretrained('modelo')

398/398 [==============================] - 8504s 21s/step - loss: 1.7544


In [88]:
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel

# Cargar el modelo y el tokenizador
model = TFGPT2LMHeadModel.from_pretrained('modelo')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Generar texto
prompt = """
    sale_order_count = fields.Integer(string='Sale Order Count', compute='_count_sale_order', readonly=True, groups="sales_team.group_sale_salesman")

    def _count_sale_order(self):
        for order in self:
            order.sale_order_count = """
input_ids = tokenizer.encode(prompt, return_tensors='tf')
output = model.generate(input_ids, max_length=110+len(prompt), do_sample=False, top_k=150)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)




All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at modelo.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.



    sale_order_count = fields.Integer(string='Sale Order Count', compute='_count_sale_order', readonly=True, groups="sales_team.group_sale_salesman")

    def _count_sale_order(self):
        for order in self:
            order.sale_order_count = 
                                                                                                                                                                                                                                                                   
